In [139]:
import z3
import sys
from z3 import *
import typing
import operator
sys.path.append("../../hw/base")
#from ..basics.asm_interp import AsmInterp
from verification_utils import CHCs, mk_int_array


stack = Array('stack', IntSort(), IntSort())
sp = Int('sp')
r0, r1 = Ints('r0 r1')
mem = Array('mem', IntSort(), IntSort())
state_vars = [stack, sp, r0, r1, mem]
Invariant: typing.TypeAlias = typing.Callable[[state_vars], bool]


def enumerateLabels(prog) -> dict:
    labels = {}
    for i, op in enumerate(prog):
        if isinstance(op, tuple):
            continue
        # label:
        labels[op[:-1]] = i
    return labels

def createCHCs(precond, postcond, prog):
    
    U = {i: Function(f"U{i}", *(v.sort() for v in [*state_vars]), BoolSort())
     for i in range(len(prog)+3)}
    l = [Implies(precond(state_vars), U[0](state_vars))]
    i = 0

    

    labels = enumerateLabels(prog)
    
    for op in prog:
        if isinstance(op, str):
            continue

        if op[0] == 'STOR' or op[0] == 'LOAD':
            if op[0] == "LOAD":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, mem[r0]), sp+1, r0, r1, mem)))
            else: # store
                l.append(Implies(U[i](state_vars), U[i+1](stack, sp, r0, r1, Store(mem, r1, r0))))
            i += 1
            continue


        # two arguments
        op1, op2 = op
        
        if op1 == 'PUSH':
            l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, int(op2)), sp+1, r0, r1, mem)))
        elif op1 == "POP":
            if op2 == 2:
                l.append(Implies(U[i](state_vars), U[i+1](stack, sp-2, stack[sp-2], stack[sp-1], mem)))
            else: # op2 == '1'
                l.append(Implies(U[i](state_vars), U[i+1](stack, sp-1, stack[sp-1], r1, mem)))
        elif op1 == "ALU":
            if op2 == "ADD":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, r0 + r1), sp+1, r0, r1, mem)))
            elif op2 == "MUL":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, r0 * r1), sp+1, r0, r1, mem)))
            elif op2 == "SUB":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, r0 - r1), sp+1, r0, r1, mem)))
            elif op2 == "NEG":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, -r0), sp+1, r0, r1, mem)))
            elif op2 == "AND":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, r0 & r1), sp+1, r0, r1, mem)))
            elif op2 == "OR":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, r0 | r1), sp+1, r0, r1, mem)))
            elif op2 == "NOT":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, ~r0), sp+1, r0, r1, mem)))
            elif op2 == "SHL":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, r0 << 1), sp+1, r0, r1, mem)))
            elif op2 == "SHR":
                l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, r0 >> 1), sp+1, r0, r1, mem)))
            elif op2 == "LT":
                l.append(Implies(And(U[i](state_vars), r0 < r1), U[i+1](Store(stack, sp, 1), sp+1, r0, r1, mem)))
                l.append(Implies(And(U[i](state_vars), r1 <= r0), U[i+1](Store(stack, sp, 0), sp+1, r0, r1, mem)))
        elif op1 == "DUP":
            n = Int(op2) if op2 else 0
            l.append(Implies(U[i](state_vars), U[i+1](Store(stack, sp, stack[sp-1-n]), sp+1, r0, r1, mem)))
        elif op1 == "JMP":
            l.append(Implies(U[i](state_vars), U[labels[op2]](state_vars)))
        elif op1 == "JNZ":
            l.append(Implies(And(U[i](state_vars), r0 != 0), U[labels[op2]](state_vars)))
            l.append(Implies(And(U[i](state_vars), r0 == 0), U[i+1](state_vars)))
        elif op1 == "JZ":
            l.append(Implies(And(U[i](state_vars), r0 == 0), U[labels[op2]](state_vars)))
            l.append(Implies(And(U[i](state_vars), r0 != 0), U[i+1](state_vars)))
        elif op1 == "RET":
            # idk
            pass
        i += 1
    l.append(Implies(U[i](state_vars), postcond(state_vars)))

    print("Labels:")
    for key, value in labels.items():
        print(f"{key}: {value}")
    
    chcs = CHCs(l)
    
    return chcs

In [140]:
# prog = [
#     ('POP', 1), # u0 => u1
#     ('STOR', ), # u1 => u2
#     ('PUSH', 1), # u2 => u3
#     ('POP', 2), # u3 => u4
#     ('STOR', ), # u4 => u5
#     'loop:', # u[5] => u[6](sdfsdf)
#     ('PUSH', 0), # u5 => u6
#     ('POP', 1), # u6 => u7
#     ('LOAD', ),
#     ('PUSH', 1),
#     ('POP', 2),
#     ('ALU', 'SUB'),
#     ('PUSH', 0),
#     ('POP', 2),
#     ('STOR', ),
#     ('JZ', 'end'),
#     ('PUSH', 2),
#     ('POP', 2),
#     ('STOR', ), # r1 = 2, r0 = 40
#     ('PUSH', 1),
#     ('POP', 1),
#     ('LOAD', ), # on stack = current max
#     ('PUSH', 2),
#     ('POP', 1),
#     ('LOAD', ), # on stack = [currentMax, current]
#     ('POP', 2), # r1 = current, r0 = currentMax
#     ('ALU', 'LT'),
#     ('POP', 1),
#     ('JZ', 'loop'),
#     ('PUSH', 2),
#     ('POP', 1),
#     ('LOAD', ),
#     ('PUSH', 1),
#     ('POP', 2), # r1 = 1
#     ('STOR', ),
#     ('JMP', 'loop'),
#     'end:',
#     ('PUSH', 1),
#     ('POP', 1),
#     ('LOAD', ),
#     ('POP', 1)
#     ]
# #precond = lambda state_vars: True
# #postcond = lambda state_vars: state_vars[2] == 434

# expected = [1,2,3,4,5,4,3,2,1,9]

# precond = lambda state_vars: And([Select(state_vars[0], i) == expected[i] for i in range(len(expected))])
# postcond = lambda state_vars: state_vars[2] == 5
# chcs = createCHCs(precond, postcond, prog)
# chcs.solve()



# prog1 = [
#     ('PUSH', 5),
#     ('PUSH', 5),
#     ('POP', 2),  # r0 = 5, r1 = 5
#     ('ALU', 'LT'),  # Should push 0 if 5 < 5 (which is false)
#     ('POP', 1), # r0 = 0
#     ('STOR', ),
#     ('PUSH', 5),
#     ('POP', 1),
#     ('LOAD', )
# ]

# precond1 = lambda state_vars: True
# postcond1 = lambda state_vars: Select(state_vars[0], state_vars[1]-1) == 0 # Check if r0 == 1
# chcs1 = createCHCs(precond1, postcond1, prog1)


program = [
    ('PUSH', 1),    # Push 1 onto stack
    ('PUSH', 0),    # Push 2 onto stack
    ('POP', 1),
    ('JNZ', 'skip'),  # Jump to label "skip"
    ('PUSH', 3),    # This should be skipped
    ('PUSH', 4),    # This should be skipped
    'skip:',   # Label to jump to
    ('POP', 1),    # Pop into r0
]

# Precondition: empty stack
precond = lambda state_vars: state_vars[1] == 0

# Postcondition: r0 should be 5 (the last value pushed)
postcond = lambda state_vars: state_vars[2] == 4
chcs = createCHCs(precond, postcond, program)
chcs.solve()

Labels:
skip: 6


U0,ν1 = 0
U1,"∃k!4 : ν1 = Store(k!4, 0, 1) ∧ ν2 = 1"
U5,"(∃k!3 : ν1 = Store(Store(k!3, 0, 1), 1, 3) ∧ ν2 = 2 ∧ ν3 = 0) ∨ (∃k!4, k!3 : (∃k!3 : 0 = k!4[1] ∧ k!4 = Store(Store(k!3, 0, 1), 1, 0) ∧ k!3 = 1 ∧ ν5 = 0) ∧ ν3 = 1 + k!3 ∧ ν2 = Store(k!4, k!3, 3))"
U7,"(∃k!4 : ¬(ν1[1] = 0) ∧ ν3 = ν1[0] ∧ ν1 = Store(Store(k!4, 0, 1), 1, 0) ∧ ν2 = 0) ∨ (∃k!3, k!2 : ((∃k!3 : ν3 = Store(Store(Store(k!3, 0, 1), 1, 3), 2, 4) ∧ k!3 = 3 ∧ k!2 = 0) ∨ (∃k!4 : ¬(k!2 = 0) ∧ k!2 = ν3[1] ∧ ν3 = Store(Store(k!4, 0, 1), 1, 0) ∧ k!3 = 1)) ∧ ν3 = -1 + k!3 ∧ ν4 = ν2[-1 + k!3]) ∨ (∃k!4 : ν3 = ν1[2] ∧ ν1 = Store(Store(Store(k!4, 0, 1), 1, 3), 2, 4) ∧ ν2 = 2)"
U4,"(∃k!3 : 0 = ν1[1] ∧ ν1 = Store(Store(k!3, 0, 1), 1, 0) ∧ ν2 = 1 ∧ ν3 = 0) ∨ (∃k!4 : 0 = ν1[1] ∧ ν1 = Store(Store(k!4, 0, 1), 1, 0) ∧ ν2 = 1) ∧ ν2 = 0"
U3,"(∃k!4 : ν3 = ν1[1] ∧ ν1 = Store(Store(k!4, 0, 1), 1, 0) ∧ ν2 = 1) ∨ (∃k!3 : (∃k!4 : ν2 = Store(Store(k!4, 0, 1), 1, 0) ∧ k!3 = 2) ∧ ν2 = -1 + k!3 ∧ ν3 = ν1[-1 + k!3])"
U6,"(∃k!3 : ν1 = Store(Store(Store(k!3, 0, 1), 1, 3), 2, 4) ∧ ν2 = 3 ∧ ν3 = 0) ∨ (∃k!4 : ν3 = ν1[1] ∧ ν1 = Store(Store(k!4, 0, 1), 1, 0) ∧ ν2 = 1) ∧ ¬(ν2 = 0) ∨ (∃k!4, k!3 : (∃k!3 : k!4 = Store(Store(k!3, 0, 1), 1, 3) ∧ k!3 = 2 ∧ ν5 = 0) ∧ ν3 = 1 + k!3 ∧ ν2 = Store(k!4, k!3, 4)) ∨ (∃k!4 : ¬(ν3 = 0) ∧ ν3 = ν1[1] ∧ ν1 = Store(Store(k!4, 0, 1), 1, 0) ∧ ν2 = 1)"
U2,"(∃k!4 : ν1 = Store(Store(k!4, 0, 1), 1, 0) ∧ ν2 = 2) ∨ (∃k!4, k!3 : (∃k!4 : k!4 = Store(k!4, 0, 1) ∧ k!3 = 1) ∧ ν3 = 1 + k!3 ∧ ν2 = Store(k!4, k!3, 0))"
